<a href="https://colab.research.google.com/github/sciencebyAJ/OWRD_Training_20240927/blob/main/Example_1_OpenET_FlowMeter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/sciencebyAJ/OWRD_Training_20240927.git

Cloning into 'OWRD_Training_20240927'...
remote: Enumerating objects: 70, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 70 (delta 21), reused 26 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (70/70), 575.62 KiB | 10.28 MiB/s, done.
Resolving deltas: 100% (21/21), done.


In [2]:
import geopandas as gpd
import glob

In [22]:
fields = gpd.read_file('/content/OWRD_Training_20240927/Shapefiles/flowmeter_shapefiles_v3/flowmeter_subset_fields_ORid_wgs84.shp')
fields_ss = fields[['OPENET_ID','Acres','SWU_ID','geometry']]

In [23]:
fields_ss.head()

,OPENET_ID,Acres,SWU_ID,geometry
0,ORx_24912,2.563825,SWU_UMAT0008,"POLYGON ((-118.44442 45.97865, -118.44443 45.9..."
1,ORx_24915,9.323391,SWU_UMAT0008,"POLYGON ((-118.44258 45.97911, -118.44187 45.9..."
2,ORx_25179,0.960043,SWU_UMAT0013,"POLYGON ((-118.38199 45.95786, -118.38197 45.9..."
3,ORx_25334,4.966444,SWU_UMAT0013,"POLYGON ((-118.38199 45.95744, -118.38187 45.9..."
4,ORx_25338,4.256518,SWU_UMAT0013,"POLYGON ((-118.38315 45.95842, -118.38204 45.9..."
